In [2]:
# Importar las bibliotecas necesarias
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

# Generar un par de claves (privada y pública)
def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )

# Cifrar un mensaje asimétrico
def cifrar_mensaje_asimetrico(mensaje, clave_publica):
    """Cifra un mensaje utilizando la clave pública."""
    mensaje_cifrado = clave_publica.encrypt(
        mensaje.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_cifrado

# Descifrar un mensaje asimétrico
def descifrar_mensaje_asimetrico(mensaje_cifrado, clave_privada):
    """Descifra un mensaje cifrado utilizando la clave privada."""
    mensaje_descifrado = clave_privada.decrypt(
        mensaje_cifrado,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_descifrado.decode()

# Validar la autenticidad del remitente
def validar_autenticidad(mensaje, firma, clave_publica):
    """Valida que el mensaje proviene del remitente verificando la firma con la clave pública."""
    try:
        clave_publica.verify(
            firma,
            mensaje.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False

# Firmar un mensaje
def firmar_mensaje(mensaje, clave_privada):
    """Genera una firma para el mensaje utilizando la clave privada."""
    firma = clave_privada.sign(
        mensaje.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return firma

mensaje = "Había una vez una joven lechera que caminaba con un cántaro de leche para vender en el mercado del pueblo."
print(f"Mensaje Original: {mensaje}")

#Generar claves, cifrar y desifrar el mensaje
generar_claves_asimetricas()
clave_privada = cargar_clave_privada()
clave_publica = cargar_clave_publica()
mensaje_cifrado = cifrar_mensaje_asimetrico(mensaje, clave_publica)
mensaje_desifrado = descifrar_mensaje_asimetrico(mensaje_cifrado, clave_privada)

# Parte 3: Validación de Autenticidad firmando con la clave privada
print("\n[Validación de Autenticidad]")
firma = firmar_mensaje(mensaje, clave_privada)
print(f"\nFirma generada: {firma}")

es_valido = validar_autenticidad(mensaje, firma, clave_publica)
print(f"\n¿El mensaje es auténtico? {'Sí' if es_valido else 'No'}")

print(f"\nMensaje cifrado: {mensaje_cifrado}")
print(f"\nMensaje descifrado: {mensaje_desifrado}")

Mensaje Original: Había una vez una joven lechera que caminaba con un cántaro de leche para vender en el mercado del pueblo.
Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.

[Validación de Autenticidad]

Firma generada: b'7\x1eav\xa0\xd1\x1a`W\x82\xff\xae\x15\xb3feS\x1a\x0f\xae\xb9\xda\xcc\x8c,\xb9n\x0e\xb4\xb7m\x97\x0e\xd0!E\x1c\xb0\xd1\xe0\r\x11/\xff\xe8\x91O\xc7[Ui\x8a\xb3\x96D**\x01tpTy&6\xde\x06\x9co\xf3\xf5\xca-k\x87\xd0\x10t>\x8d\xee\x82\x97\x82\x10)\x0cL\x05\x95\xc2\xf4\xb5*Z\x96\xcc\xd7n<\x1f\xf5\x11z\xae{\xdd\x83x\x03\x8e\xa0\x0cBw\xaf<\x97\xffWi\x92P\xde\xe3y\x80\xa2\xc8c\xbc\xad1\xc6\xa06\x13)\xa5\xa0\xb8\x11\xab\xcf\xddv\xd7m?\xe2\xf6LF\xb9\xbb\x80\xa3\x9c\xfbab\xda\xa2\x05\xc6\x80\xdc\xf2\xf9\t\x18G\x0b\xde\xa0H\xfa\x8a1\xae\x14=)\x9ff\x94\xe3\xb3X\xc4\x846\xf8\xe4p?F\r\xab!W\xc5\x08\xaf\xb7\xa0\xc0\xb0g\xd3z\x12\xf4\x9fk\xc8\xd0y\x88\xbb\xe5\xfd.\x88\xcfM\x99\xe9\x8c\x8265s\x8fK\xd9\xbf\xa9^H2\xa6cA2_e\x93\x8d\xc1\xf5\x05\xd9p\xf2\x

### Actividad de reflexión
Discutir con los estudiantes:
- ¿Qué ventajas tiene el cifrado asimétrico?
-- Tener una capa adicional de seguridad en comparación con el cifrado simetrico, ya que sin la clave privada no es posible desifrar el mensaje.
- ¿Qué posibles fallas de seguridad pueden existir?
-- El robo de la llave privada.
-- El compartir la llave privada por un medio vulnerable.
- ¿Qué sucede si yo pierdo mi clave privada?
-- No podría desicrar el mensaje.
-¿Qué sucede si la comparto?
-- El receptor podra desifrar el mensaje.
-- Si el medio es vulnerable podrían robar la llave y desifrar el mensaje.
- Intente codificar la fábula de la lechera u otro texto largo ¿Por qué da un error?
-- RSA no está diseñado para cifrar grandes cantidades de datos directamente. Tiene una limitación de alrededor de 245 bytes.

¿Cómo se puede arreglar este error para compartir mensajes cifrados?
-- Utilizando un algoritmo hibrido con el cifrado simetrico y asimetrico
- Comparta mensajes largos cifrados entre los estudiantes y que ellos validen la autencididad del remitente

In [1]:
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
import base64

# Paso 1: Generar par de claves RSA
def generar_claves_asimetricas():
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    with open("clave_privada.pem", "wb") as f:
        f.write(clave_privada.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    with open("clave_publica.pem", "wb") as f:
        f.write(clave_publica.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

def cargar_clave_privada():
    with open("clave_privada.pem", "rb") as f:
        return serialization.load_pem_private_key(f.read(), password=None)

def cargar_clave_publica():
    with open("clave_publica.pem", "rb") as f:
        return serialization.load_pem_public_key(f.read())

# Paso 2: Cifrado híbrido
def cifrar_mensaje_hibrido(mensaje, clave_publica_rsa):
    # 1. Generar clave simétrica
    clave_simetrica = Fernet.generate_key()
    fernet = Fernet(clave_simetrica)

    # 2. Cifrar mensaje con Fernet
    mensaje_cifrado = fernet.encrypt(mensaje.encode())

    # 3. Cifrar clave simétrica con clave pública
    clave_simetrica_cifrada = clave_publica_rsa.encrypt(
        clave_simetrica,
        padding.OAEP(
            mgf=padding.MGF1(hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return mensaje_cifrado, clave_simetrica_cifrada

# Paso 3: Descifrado híbrido
def descifrar_mensaje_hibrido(mensaje_cifrado, clave_simetrica_cifrada, clave_privada_rsa):
    clave_simetrica = clave_privada_rsa.decrypt(
        clave_simetrica_cifrada,
        padding.OAEP(
            mgf=padding.MGF1(hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    fernet = Fernet(clave_simetrica)
    mensaje_descifrado = fernet.decrypt(mensaje_cifrado).decode()
    return mensaje_descifrado

# Firmar mensaje
def firmar_mensaje(mensaje, clave_privada):
    firma = clave_privada.sign(
        mensaje.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return firma

# Validar firma
def validar_autenticidad(mensaje, firma, clave_publica):
    try:
        clave_publica.verify(
            firma,
            mensaje.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False

mensaje = "Había una vez una joven lechera que caminaba con un cántaro de leche para vender en el mercado del pueblo. Mientras caminaba pensaba en todas las cosas que haría con el dinero de la venta: Cuando me paguen se dijo, compraré de inmediato unas gallinas, estas gallinas pondrán muchísimos huevos y los venderé en el mercado. Con el dinero de los huevos me compraré un vestido y zapatos muy elegantes. Luego, iré a la feria y como luciré tan hermosa, todos los chicos querrán acercarse a hablar conmigo. Por andar distraída con sus pensamientos, la lechera tropezó con una piedra y el cántaro se rompió derramando toda la leche. Con el cántaro destrozado se fueron las gallinas y los huevos; también el vestido y los zapatos"
print("Mensaje Original:")
print(mensaje)

# Generar claves y cifrar
generar_claves_asimetricas()
clave_privada = cargar_clave_privada()
clave_publica = cargar_clave_publica()

# Cifrado híbrido
mensaje_cifrado, clave_simetrica_cifrada = cifrar_mensaje_hibrido(mensaje, clave_publica)
mensaje_descifrado = descifrar_mensaje_hibrido(mensaje_cifrado, clave_simetrica_cifrada, clave_privada)

# Firma y verificación
firma = firmar_mensaje(mensaje, clave_privada)
autentico = validar_autenticidad(mensaje, firma, clave_publica)

print("\nMensaje Cifrado (base64):")
print(base64.b64encode(mensaje_cifrado).decode())

print("\nMensaje Descifrado:")
print(mensaje_descifrado)

print("\nFirma generada:", base64.b64encode(firma).decode())
print(f"\n¿El mensaje es auténtico? {'Sí' if autentico else 'No'}")


Mensaje Original:
Había una vez una joven lechera que caminaba con un cántaro de leche para vender en el mercado del pueblo. Mientras caminaba pensaba en todas las cosas que haría con el dinero de la venta: Cuando me paguen se dijo, compraré de inmediato unas gallinas, estas gallinas pondrán muchísimos huevos y los venderé en el mercado. Con el dinero de los huevos me compraré un vestido y zapatos muy elegantes. Luego, iré a la feria y como luciré tan hermosa, todos los chicos querrán acercarse a hablar conmigo. Por andar distraída con sus pensamientos, la lechera tropezó con una piedra y el cántaro se rompió derramando toda la leche. Con el cántaro destrozado se fueron las gallinas y los huevos; también el vestido y los zapatos

Mensaje Cifrado (base64):
Z0FBQUFBQm9VQnlSLVBQbmdfcXZOSzlCNEhYWXQ3ZklQUHZKZnliRXRrOU5XQ2Z2V1BqTjlXeU5xYVA2MjZCS05JeURiZGthZFZLT014dGxERG5Wcy1EY1NhNHlsNHJpLS11V3A4eWRNN05Xd0JXTEluRHNYMHM5MndSNlItNkZpdHU4c3VyRExkZGtBdFJFeUtjNGlDZE5vVFdIc1UyM2xQbXRTNkRDQW1fNnlHV1